# Simulation
## Group project
### 42180 Quantitative modelling of behaviour

------

**Group 11** <br>

------

Camille Babin <br>
Martine Grangé <br>
Mikkel Johansen <br>
Kyriakos Michailidis <br>
Jiri Spitzer

------

Import libraries and enable inline plotting

In [63]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import warnings
from collections import Counter
from re import search as Search
from IPython.display import display

In [64]:
%matplotlib inline

Disable warnings

In [65]:
warnings.filterwarnings('ignore')

Load data

In [66]:
data_path = r'.\data\Elisa_thesis_data_noPC.csv'

In [67]:
df = pd.read_csv(data_path)
df.head(2)

,VAR1,ID,CHOICE,Alt_Description_ICV1,CarClass_ICV1,CarType_ICV1,Purchaseprice_ICV1,Operationcost_ICV1,Yearlycost__ICV1,Range_ICV1,Acceleration_ICV1,Size_ICV1,CO2_ICV1,Charginginfrastructure_ICV1,Chargingspeed_ICV1,Homedistnearestcharger_ICV1,Homechargeravailability_ICV1,Alt_Description_ICV2,CarClass_ICV2,CarType_ICV2,Purchaseprice_ICV2,Operationcost_ICV2,Yearlycost__ICV2,Range_ICV2,Acceleration_ICV2,...,q16_5_resp,q16_6_resp,q16_7_resp,q16_8_resp,q16_9_resp,q16_10_resp,q16_11_resp,q16_12_resp,q16_13_resp,q16_14_resp,q16_15_resp,q16_16_resp,q16_17_resp,q16_18_resp,q16_19_resp,q16_20_resp,q16_21_resp,q16_22_resp,q16_23_resp,q16_24_resp,House_single,House_couple,House_children,House_couple_children,House_couple_nochildren
0,0,1,1,1,2,1,200000,1.11,4300,1097,14,2,138,0,0,0,0,2,4,1,430000,1.25,7267,735,12,...,5,5,2,2,4,3,5,3,3,4,4,3,3,3,4,1,5,4,2,5,1,0,0,0,0
1,1,1,3,1,2,1,185000,0.88,4300,898,12,3,156,0,0,0,0,2,4,1,350000,1.00,7267,997,14,...,5,5,2,2,4,3,5,3,3,4,4,3,3,3,4,1,5,4,2,5,1,0,0,0,0


#### Data preparation

Deleting rows with unknown choice.

In [68]:
df_mod=df[df['CHOICE']>0].copy(deep=True)
df_mod.reset_index();

Saving number of rows and columns.

In [69]:
n,m=df_mod.shape

In [70]:
av_cho=[]
for i in df_mod.index:
    av_cho.append([])
    if df_mod['CarClass_ICV1'].loc[i]>0: 
        av_cho[-1].append(df_mod['CarClass_ICV1'].loc[i])
    if df_mod['CarClass_ICV2'].loc[i]>0:
        av_cho[-1].append(df_mod['CarClass_ICV2'].loc[i])
    if df_mod['CarClass_BEV1'].loc[i]>0:
        av_cho[-1].append(df_mod['CarClass_BEV1'].loc[i]+6)
    if df_mod['CarClass_BEV2'].loc[i]>0:
        av_cho[-1].append(df_mod['CarClass_BEV2'].loc[i]+6)
    if df_mod['CarClass_PHEV1'].loc[i]>0: 
        av_cho[-1].append(df_mod['CarClass_PHEV1'].loc[i]+12)
    if df_mod['CarClass_PHEV2'].loc[i]>0:
        av_cho[-1].append(df_mod['CarClass_PHEV2'].loc[i]+12)

In [71]:
avail=['AV1','AV2','AV3','AV4','AV5','AV6','AV7','AV8','AV9','AV10','AV11','AV12','AV13','AV14','AV15','AV16','AV17','AV18']
for i in range(18):
    df_mod[avail[i]]=np.zeros(n,int)
    for j in range(n):
        if i+1 in av_cho[j]:
            df_mod[avail[i]].loc[df_mod.index[j]]=1

In [72]:
df_mod['Gender_Dummy']=np.zeros(n,int)
df_mod['Dummy_Age2']=np.zeros(n,int)
df_mod['Dummy_Age3']=np.zeros(n,int)
df_mod['Dummy_Age4']=np.zeros(n,int)
df_mod['Dummy_Age5']=np.zeros(n,int)
for j in range(n):
    df_mod['Gender_Dummy'].loc[df_mod.index[j]]=df_mod['c_Gender'].loc[df_mod.index[j]]-1
    if df_mod['c_Age_cat'].loc[df_mod.index[j]]==2:
        df_mod['Dummy_Age2'].loc[df_mod.index[j]]=1
    if df_mod['c_Age_cat'].loc[df_mod.index[j]]==3:
        df_mod['Dummy_Age3'].loc[df_mod.index[j]]=1
    if df_mod['c_Age_cat'].loc[df_mod.index[j]]==4:
        df_mod['Dummy_Age4'].loc[df_mod.index[j]]=1
    if df_mod['c_Age_cat'].loc[df_mod.index[j]]==5:
        df_mod['Dummy_Age5'].loc[df_mod.index[j]]=1

In [73]:
columns=["Size_ICV_MINI","Size_ICV_LIL","Size_ICV_MEL","Size_ICV_STOR","Size_ICV_PREM","Size_ICV_LUK",
         "Size_BEV_MINI","Size_BEV_LIL","Size_BEV_MEL","Size_BEV_STOR","Size_BEV_PREM","Size_BEV_LUK",
         "Size_PHEV_MINI","Size_PHEV_LIL","Size_PHEV_MEL","Size_PHEV_STOR","Size_PHEV_PREM","Size_PHEV_LUK"]
for column in columns:
    df_mod[column+"_Large_Dummy"]=np.zeros(n,int)
    for j in range(n):
        if df_mod[column].loc[df_mod.index[j]]>2:
            df_mod[column+"_Large_Dummy"].loc[df_mod.index[j]]=1
        else:
            df_mod[column+"_Large_Dummy"].loc[df_mod.index[j]]=0

In [74]:
pd.options.display.max_columns = 500

In [75]:
df_mod.head(5)

,VAR1,ID,CHOICE,Alt_Description_ICV1,CarClass_ICV1,CarType_ICV1,Purchaseprice_ICV1,Operationcost_ICV1,Yearlycost__ICV1,Range_ICV1,Acceleration_ICV1,Size_ICV1,CO2_ICV1,Charginginfrastructure_ICV1,Chargingspeed_ICV1,Homedistnearestcharger_ICV1,Homechargeravailability_ICV1,Alt_Description_ICV2,CarClass_ICV2,CarType_ICV2,Purchaseprice_ICV2,Operationcost_ICV2,Yearlycost__ICV2,Range_ICV2,Acceleration_ICV2,Size_ICV2,CO2_ICV2,Charginginfrastructure_ICV2,Chargingspeed_ICV2,Homedistnearestcharger_ICV2,Homechargeravailability_ICV2,Alt_Description_BEV1,CarClass_BEV1,CarType_BEV1,Purchaseprice_BEV1,Operationcost_BEV1,Yearlycost__BEV1,Range_BEV1,Acceleration_BEV1,Size_BEV1,CO2_BEV1,Charginginfrastructure_BEV1,Chargingspeed_BEV1,Homedistnearestcharger_BEV1,Homechargeravailability_BEV1,Alt_Description_BEV2,CarClass_BEV2,CarType_BEV2,Purchaseprice_BEV2,Operationcost_BEV2,Yearlycost__BEV2,Range_BEV2,Acceleration_BEV2,Size_BEV2,CO2_BEV2,Charginginfrastructure_BEV2,Chargingspeed_BEV2,Homedistnearestcharger_BEV2,Homechargeravailability_BEV2,Alt_Description_PHEV1,CarClass_PHEV1,CarType_PHEV1,Purchaseprice_PHEV1,Operationcost_PHEV1,Yearlycost__PHEV1,Range_PHEV1,Acceleration_PHEV1,Size_PHEV1,CO2_PHEV1,Charginginfrastructure_PHEV1,Chargingspeed_PHEV1,Homedistnearestcharger_PHEV1,Homechargeravailability_PHEV1,Alt_Description_PHEV2,CarClass_PHEV2,CarType_PHEV2,Purchaseprice_PHEV2,Operationcost_PHEV2,Yearlycost__PHEV2,Range_PHEV2,Acceleration_PHEV2,Size_PHEV2,CO2_PHEV2,Charginginfrastructure_PHEV2,Chargingspeed_PHEV2,Homedistnearestcharger_PHEV2,Homechargeravailability_PHEV2,Purchaseprice_ICV_MINI,Operationcost_ICV_MINI,Yearlycost__ICV_MINI,Range_ICV_MINI,Acceleration_ICV_MINI,Size_ICV_MINI,CO2_ICV_MINI,Charginginfrastructure_ICV_MINI,Chargingspeed_ICV_MINI,Homedistnearestcharger_ICV_MINI,Homechargeravailability_ICV_MINI,Purchaseprice_ICV_LIL,Operationcost_ICV_LIL,Yearlycost__ICV_LIL,Range_ICV_LIL,Acceleration_ICV_LIL,Size_ICV_LIL,CO2_ICV_LIL,Charginginfrastructure_ICV_LIL,Chargingspeed_ICV_LIL,Homedistnearestcharger_ICV_LIL,Homechargeravailability_ICV_LIL,Purchaseprice_ICV_MEL,Operationcost_ICV_MEL,Yearlycost__ICV_MEL,Range_ICV_MEL,Acceleration_ICV_MEL,Size_ICV_MEL,CO2_ICV_MEL,Charginginfrastructure_ICV_MEL,Chargingspeed_ICV_MEL,Homedistnearestcharger_ICV_MEL,Homechargeravailability_ICV_MEL,Purchaseprice_ICV_STOR,Operationcost_ICV_STOR,Yearlycost__ICV_STOR,Range_ICV_STOR,Acceleration_ICV_STOR,Size_ICV_STOR,CO2_ICV_STOR,Charginginfrastructure_ICV_STOR,Chargingspeed_ICV_STOR,Homedistnearestcharger_ICV_STOR,Homechargeravailability_ICV_STOR,Purchaseprice_ICV_PREM,Operationcost_ICV_PREM,Yearlycost__ICV_PREM,Range_ICV_PREM,Acceleration_ICV_PREM,Size_ICV_PREM,CO2_ICV_PREM,Charginginfrastructure_ICV_PREM,Chargingspeed_ICV_PREM,Homedistnearestcharger_ICV_PREM,Homechargeravailability_ICV_PREM,Purchaseprice_ICV_LUK,Operationcost_ICV_LUK,Yearlycost__ICV_LUK,Range_ICV_LUK,Acceleration_ICV_LUK,Size_ICV_LUK,CO2_ICV_LUK,Charginginfrastructure_ICV_LUK,Chargingspeed_ICV_LUK,Homedistnearestcharger_ICV_LUK,Homechargeravailability_ICV_LUK,Purchaseprice_BEV_MINI,Operationcost_BEV_MINI,Yearlycost__BEV_MINI,Range_BEV_MINI,Acceleration_BEV_MINI,Size_BEV_MINI,CO2_BEV_MINI,Charginginfrastructure_BEV_MINI,Chargingspeed_BEV_MINI,Homedistnearestcharger_BEV_MINI,Homechargeravailability_BEV_MINI,Purchaseprice_BEV_LIL,Operationcost_BEV_LIL,Yearlycost__BEV_LIL,Range_BEV_LIL,Acceleration_BEV_LIL,Size_BEV_LIL,CO2_BEV_LIL,Charginginfrastructure_BEV_LIL,Chargingspeed_BEV_LIL,Homedistnearestcharger_BEV_LIL,Homechargeravailability_BEV_LIL,Purchaseprice_BEV_MEL,Operationcost_BEV_MEL,Yearlycost__BEV_MEL,Range_BEV_MEL,Acceleration_BEV_MEL,Size_BEV_MEL,CO2_BEV_MEL,Charginginfrastructure_BEV_MEL,Chargingspeed_BEV_MEL,Homedistnearestcharger_BEV_MEL,Homechargeravailability_BEV_MEL,Purchaseprice_BEV_STOR,Operationcost_BEV_STOR,Yearlycost__BEV_STOR,Range_BEV_STOR,Acceleration_BEV_STOR,Size_BEV_STOR,CO2_BEV_STOR,Charginginfrastructure_BEV_STOR,Chargingspeed_BEV_STOR,Homedistnearestcharger_BEV_STOR,Home

In [76]:
pd.options.display.max_columns = 50

#### Utility functions

In [77]:
'''
V1 = ASC_1_RND + B_PP*Purchaseprice_ICV_MINI + B_OC*Operationcost_ICV_MINI + B_YC*Yearlycost__ICV_MINI + B_Ra*Range_ICV_MINI + B_Acc*Acceleration_ICV_MINI + B_SizeLarge*Size_ICV_MINI_Large_Dummy + B_Co2*CO2_ICV_MINI + B_ChInf*Charginginfrastructure_ICV_MINI + B_ChSpeed*Chargingspeed_ICV_MINI + B_Dist*Homedistnearestcharger_ICV_MINI + B_HomeCh*Homechargeravailability_ICV_MINI
V2 = ASC_2_RND + B_PP*Purchaseprice_ICV_LIL + B_OC*Operationcost_ICV_LIL + B_YC*Yearlycost__ICV_LIL + B_Ra*Range_ICV_LIL + B_Acc*Acceleration_ICV_LIL + B_SizeLarge*Size_ICV_LIL_Large_Dummy + B_Co2*CO2_ICV_LIL + B_ChInf*Charginginfrastructure_ICV_LIL + B_ChSpeed*Chargingspeed_ICV_LIL + B_Dist*Homedistnearestcharger_ICV_LIL + B_HomeCh*Homechargeravailability_ICV_LIL
V3 = ASC_3_RND + B_PP*Purchaseprice_ICV_MEL + B_OC*Operationcost_ICV_MEL + B_YC*Yearlycost__ICV_MEL + B_Ra*Range_ICV_MEL + B_Acc*Acceleration_ICV_MEL + B_SizeLarge*Size_ICV_MEL_Large_Dummy + B_Co2*CO2_ICV_MEL + B_ChInf*Charginginfrastructure_ICV_MEL + B_ChSpeed*Chargingspeed_ICV_MEL + B_Dist*Homedistnearestcharger_ICV_MEL + B_HomeCh*Homechargeravailability_ICV_MEL
V4 = ASC_4_RND + B_PP*Purchaseprice_ICV_STOR + B_OC*Operationcost_ICV_STOR + B_YC*Yearlycost__ICV_STOR + B_Ra*Range_ICV_STOR + B_Acc*Acceleration_ICV_STOR + B_SizeLarge*Size_ICV_STOR_Large_Dummy + B_Co2*CO2_ICV_STOR + B_ChInf*Charginginfrastructure_ICV_STOR + B_ChSpeed*Chargingspeed_ICV_STOR + B_Dist*Homedistnearestcharger_ICV_STOR + B_HomeCh*Homechargeravailability_ICV_STOR
V5 = ASC_5_RND + B_PP*Purchaseprice_ICV_PREM + B_OC*Operationcost_ICV_PREM + B_YC*Yearlycost__ICV_PREM + B_Ra*Range_ICV_PREM + B_Acc*Acceleration_ICV_PREM + B_SizeLarge*Size_ICV_PREM_Large_Dummy + B_Co2*CO2_ICV_PREM + B_ChInf*Charginginfrastructure_ICV_PREM + B_ChSpeed*Chargingspeed_ICV_PREM + B_Dist*Homedistnearestcharger_ICV_PREM + B_HomeCh*Homechargeravailability_ICV_PREM
V6 = ASC_6_RND + B_PP*Purchaseprice_ICV_LUK + B_OC*Operationcost_ICV_LUK + B_YC*Yearlycost__ICV_LUK + B_Ra*Range_ICV_LUK + B_Acc*Acceleration_ICV_LUK + B_SizeLarge*Size_ICV_LUK_Large_Dummy + B_Co2*CO2_ICV_LUK + B_ChInf*Charginginfrastructure_ICV_LUK + B_ChSpeed*Chargingspeed_ICV_LUK + B_Dist*Homedistnearestcharger_ICV_LUK + B_HomeCh*Homechargeravailability_ICV_LUK

V7 = ASC_7_RND + B_PP*Purchaseprice_BEV_MINI + B_OC*Operationcost_BEV_MINI + B_YC*Yearlycost__BEV_MINI + B_Ra*Range_BEV_MINI + B_Acc*Acceleration_BEV_MINI + B_SizeLarge*Size_BEV_MINI_Large_Dummy + B_Co2*CO2_BEV_MINI + B_ChInf*Charginginfrastructure_BEV_MINI + B_ChSpeed*Chargingspeed_BEV_MINI + B_Dist*Homedistnearestcharger_BEV_MINI + B_HomeCh*Homechargeravailability_BEV_MINI
V8 = ASC_8_RND + B_PP*Purchaseprice_BEV_LIL + B_OC*Operationcost_BEV_LIL + B_YC*Yearlycost__BEV_LIL + B_Ra*Range_BEV_LIL + B_Acc*Acceleration_BEV_LIL + B_SizeLarge*Size_BEV_LIL_Large_Dummy + B_Co2*CO2_BEV_LIL + B_ChInf*Charginginfrastructure_BEV_LIL + B_ChSpeed*Chargingspeed_BEV_LIL + B_Dist*Homedistnearestcharger_BEV_LIL + B_HomeCh*Homechargeravailability_BEV_LIL
V9 = ASC_9_RND + B_PP*Purchaseprice_BEV_MEL + B_OC*Operationcost_BEV_MEL + B_YC*Yearlycost__BEV_MEL + B_Ra*Range_BEV_MEL + B_Acc*Acceleration_BEV_MEL + B_SizeLarge*Size_BEV_MEL_Large_Dummy + B_Co2*CO2_BEV_MEL + B_ChInf*Charginginfrastructure_BEV_MEL + B_ChSpeed*Chargingspeed_BEV_MEL + B_Dist*Homedistnearestcharger_BEV_MEL + B_HomeCh*Homechargeravailability_BEV_MEL
V10 = ASC_10_RND + B_PP*Purchaseprice_BEV_STOR + B_OC*Operationcost_BEV_STOR + B_YC*Yearlycost__BEV_STOR + B_Ra*Range_BEV_STOR + B_Acc*Acceleration_BEV_STOR + B_SizeLarge*Size_BEV_STOR_Large_Dummy + B_Co2*CO2_BEV_STOR + B_ChInf*Charginginfrastructure_BEV_STOR + B_ChSpeed*Chargingspeed_BEV_STOR + B_Dist*Homedistnearestcharger_BEV_STOR + B_HomeCh*Homechargeravailability_BEV_STOR
V11 = ASC_11_RND + B_PP*Purchaseprice_BEV_PREM + B_OC*Operationcost_BEV_PREM + B_YC*Yearlycost__BEV_PREM + B_Ra*Range_BEV_PREM + B_Acc*Acceleration_BEV_PREM + B_SizeLarge*Size_BEV_PREM_Large_Dummy + B_Co2*CO2_BEV_PREM + B_ChInf*Charginginfrastructure_BEV_PREM + B_ChSpeed*Chargingspeed_BEV_PREM + B_Dist*Homedistnearestcharger_BEV_PREM + B_HomeCh*Homechargeravailability_BEV_PREM
V12 = ASC_12_RND + B_PP*Purchaseprice_BEV_LUK + B_OC*Operationcost_BEV_LUK + B_YC*Yearlycost__BEV_LUK + B_Ra*Range_BEV_LUK + B_Acc*Acceleration_BEV_LUK + B_SizeLarge*Size_BEV_LUK_Large_Dummy + B_Co2*CO2_BEV_LUK + B_ChInf*Charginginfrastructure_BEV_LUK + B_ChSpeed*Chargingspeed_BEV_LUK + B_Dist*Homedistnearestcharger_BEV_LUK + B_HomeCh*Homechargeravailability_BEV_LUK

V13 = ASC_13_RND + B_PP*Purchaseprice_PHEV_MINI + B_OC*Operationcost_PHEV_MINI + B_YC*Yearlycost__PHEV_MINI + B_Ra*Range_PHEV_MINI + B_Acc*Acceleration_PHEV_MINI + B_SizeLarge*Size_PHEV_MINI_Large_Dummy + B_Co2*CO2_PHEV_MINI + B_ChInf*Charginginfrastructure_PHEV_MINI + B_ChSpeed*Chargingspeed_PHEV_MINI + B_Dist*Homedistnearestcharger_PHEV_MINI + B_HomeCh*Homechargeravailability_PHEV_MIN
V14 = ASC_14_RND + B_PP*Purchaseprice_PHEV_LIL + B_OC*Operationcost_PHEV_LIL + B_YC*Yearlycost__PHEV_LIL + B_Ra*Range_PHEV_LIL + B_Acc*Acceleration_PHEV_LIL + B_SizeLarge*Size_PHEV_LIL_Large_Dummy + B_Co2*CO2_PHEV_LIL + B_ChInf*Charginginfrastructure_PHEV_LIL + B_ChSpeed*Chargingspeed_PHEV_LIL + B_Dist*Homedistnearestcharger_PHEV_LIL + B_HomeCh*Homechargeravailability_PHEV_LIL
V15 = ASC_15_RND + B_PP*Purchaseprice_PHEV_MEL + B_OC*Operationcost_PHEV_MEL + B_YC*Yearlycost__PHEV_MEL + B_Ra*Range_PHEV_MEL + B_Acc*Acceleration_PHEV_MEL + B_SizeLarge*Size_PHEV_MEL_Large_Dummy + B_Co2*CO2_PHEV_MEL + B_ChInf*Charginginfrastructure_PHEV_MEL + B_ChSpeed*Chargingspeed_PHEV_MEL + B_Dist*Homedistnearestcharger_PHEV_MEL + B_HomeCh*Homechargeravailability_PHEV_MEL
V16 = ASC_16_RND + B_PP*Purchaseprice_PHEV_STOR + B_OC*Operationcost_PHEV_STOR + B_YC*Yearlycost__PHEV_STOR + B_Ra*Range_PHEV_STOR + B_Acc*Acceleration_PHEV_STOR + B_SizeLarge*Size_PHEV_STOR_Large_Dummy + B_Co2*CO2_PHEV_STOR + B_ChInf*Charginginfrastructure_PHEV_STOR + B_ChSpeed*Chargingspeed_PHEV_STOR + B_Dist*Homedistnearestcharger_PHEV_STOR + B_HomeCh*Homechargeravailability_PHEV_STO
V17 = ASC_17_RND + B_PP*Purchaseprice_PHEV_PREM + B_OC*Operationcost_PHEV_PREM + B_YC*Yearlycost__PHEV_PREM + B_Ra*Range_PHEV_PREM + B_Acc*Acceleration_PHEV_PREM + B_SizeLarge*Size_PHEV_PREM_Large_Dummy + B_Co2*CO2_PHEV_PREM + B_ChInf*Charginginfrastructure_PHEV_PREM + B_ChSpeed*Chargingspeed_PHEV_PREM + B_Dist*Homedistnearestcharger_PHEV_PREM + B_HomeCh*Homechargeravailability_PHEV_PRE
V18 = ASC_18 + B_PP*Purchaseprice_PHEV_LUK + B_OC*Operationcost_PHEV_LUK + B_YC*Yearlycost__PHEV_LUK + B_Ra*Range_PHEV_LUK + B_Acc*Acceleration_PHEV_LUK + B_SizeLarge*Size_PHEV_LUK_Large_Dummy + B_Co2*CO2_PHEV_LUK + B_ChInf*Charginginfrastructure_PHEV_LUK + B_ChSpeed*Chargingspeed_PHEV_LUK + B_Dist*Homedistnearestcharger_PHEV_LUK + B_HomeCh*Homechargeravailability_PHEV_LUK
''';

Parameters

In [78]:
ASC_1 = -6.17
ASC_1_S = 2.86
ASC_2 = -6.51
ASC_2_S = 3.27
ASC_3 = -5.41
ASC_3_S = 2.81
ASC_4 = -5.9
ASC_4_S = 2.86
ASC_5 = -5.67
ASC_5_S = 3.11
ASC_6 = -5.92
ASC_6_S = 5.38
ASC_7 = -5.57
ASC_7_S = 2.34
ASC_8 = -6.15
ASC_8_S = 2.04
ASC_9 = -5.01
ASC_9_S = 2.02
ASC_10 = -5.12
ASC_10_S = 2.04
ASC_11 = -5.02
ASC_11_S = 2.46
ASC_12 = -3.14
ASC_12_S = 2.37
ASC_13 = -4.7
ASC_13_S = 2.29
ASC_14 = -5.29
ASC_14_S = 2.75
ASC_15 = -3.52
ASC_15_S = 2.73
ASC_16 = -3.59
ASC_16_S = 2.32
ASC_17 = -3.88
ASC_17_S = 2.6

B_Acc = -0.0345
B_ChInf = -0.000261
B_ChSpeed = 0.0039
B_Co2 = -0.00325
B_Dist = -0.00106
B_HomeCh = 0.137
B_OC = -0.608
B_PP = -6.5 * 10 ** -6
B_Ra = 0.0019
B_SizeLarge = 0.23
B_YC = -9.55 * 10 ** -5

Utility functions ICEV

In [79]:
def V1(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return ASC_1+ASC_1_S*Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V2(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return ASC_2+ASC_2_S*Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V3(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return ASC_3+ASC_3_S*Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V4(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return ASC_4+ASC_4_S*Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V5(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return ASC_5+ASC_5_S*Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V6(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return ASC_6+ASC_6_S*Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

Utility functions BEV

In [80]:
def V7(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return ASC_7+ASC_7_S*Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V8(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return ASC_8+ASC_8_S*Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V9(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return ASC_9+ASC_9_S*Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V10(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return ASC_10+ASC_10_S*Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V11(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return ASC_11+ASC_11_S*Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V12(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return ASC_12+ASC_12_S*Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

Utility functions PHEV

In [81]:
def V13(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return ASC_13+ASC_13_S*Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V14(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return ASC_14+ASC_14_S*Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V15(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return ASC_15+ASC_15_S*Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V16(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return ASC_16+ASC_16_S*Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V17(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return ASC_17+ASC_17_S*Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V18(Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

#### Probability function

In [82]:
def prob(mu, *args):
    total = 0.0
    for arg in args:
        if arg[0] == 1:
            total += np.exp(arg[1] * mu)
    prob = []
    for arg in args:
        if arg[0] == 1:
            prob.append(np.exp(mu*arg[1]) / total)
        else:
            prob.append(0)
    return prob

'''
def prob(mu, *args):
    total = 0.0
    for arg in args:
        total += np.exp(arg * mu)
    return [np.exp(mu*x) / total for x in args]
''';

#### Simulation

In [83]:
def sim(probabilityVector):
    r = np.random.random()
    n = 0
    for i,value in enumerate(probabilityVector):
        n += value
        if r <= n:
            break
    return i

#### Mapping

In [84]:
mapping = {
    0:"ICEV_1",1:"ICEV_2",2:"ICEV_3",3:"ICEV_4",4:"ICEV_5",5:"ICEV_6",
    6:"BEV_1",7:"BEV_2",8:"BEV_3",9:"BEV_4",10:"BEV_5",11:"BEV_6",
    12:"PHEV_1",13:"PHEV_2",14:"PHEV_3",15:"PHEV_4",16:"PHEV_5",17:"PHEV_6"
}

#### Computing market share estimates

Calculating probabilities

In [86]:
mu = 1

probMatrix = []
for index, row in df_mod.iterrows():
    v1 = (row['AV1'], V1(np.random.normal(),row['Purchaseprice_ICV_MINI'],row['Operationcost_ICV_MINI'],row['Yearlycost__ICV_MINI'],row['Range_ICV_MINI'],row['Acceleration_ICV_MINI'],row['Size_ICV1'],row['CO2_ICV_MINI'],row['Charginginfrastructure_ICV_MINI'],row['Chargingspeed_ICV_MINI'],row['Homedistnearestcharger_ICV_MINI'],row['Homechargeravailability_ICV_MINI']))
    v2 = (row['AV2'], V2(np.random.normal(),row['Purchaseprice_ICV_LIL'],row['Operationcost_ICV_LIL'],row['Yearlycost__ICV_LIL'],row['Range_ICV_LIL'],row['Acceleration_ICV_LIL'],row['Size_ICV_LIL'],row['CO2_ICV_LIL'],row['Charginginfrastructure_ICV_LIL'],row['Chargingspeed_ICV_LIL'],row['Homedistnearestcharger_ICV_LIL'],row['Homechargeravailability_ICV_LIL']))
    v3 = (row['AV3'], V3(np.random.normal(),row['Purchaseprice_ICV_MEL'],row['Operationcost_ICV_MEL'],row['Yearlycost__ICV_MEL'],row['Range_ICV_MEL'],row['Acceleration_ICV_MEL'],row['Size_ICV_MEL'],row['CO2_ICV_MEL'],row['Charginginfrastructure_ICV_MEL'],row['Chargingspeed_ICV_MEL'],row['Homedistnearestcharger_ICV_MEL'],row['Homechargeravailability_ICV_MEL']))
    v4 = (row['AV4'], V4(np.random.normal(),row['Purchaseprice_ICV_STOR'],row['Operationcost_ICV_STOR'],row['Yearlycost__ICV_STOR'],row['Range_ICV_STOR'],row['Acceleration_ICV_STOR'],row['Size_ICV_STOR'],row['CO2_ICV_STOR'],row['Charginginfrastructure_ICV_STOR'],row['Chargingspeed_ICV_STOR'],row['Homedistnearestcharger_ICV_STOR'],row['Homechargeravailability_ICV_STOR']))
    v5 = (row['AV5'], V5(np.random.normal(),row['Purchaseprice_ICV_PREM'],row['Operationcost_ICV_PREM'],row['Yearlycost__ICV_PREM'],row['Range_ICV_PREM'],row['Acceleration_ICV_PREM'],row['Size_ICV_PREM'],row['CO2_ICV_PREM'],row['Charginginfrastructure_ICV_PREM'],row['Chargingspeed_ICV_PREM'],row['Homedistnearestcharger_ICV_PREM'],row['Homechargeravailability_ICV_PREM']))
    v6 = (row['AV6'], V6(np.random.normal(),row['Purchaseprice_ICV_LUK'],row['Operationcost_ICV_LUK'],row['Yearlycost__ICV_LUK'],row['Range_ICV_LUK'],row['Acceleration_ICV_LUK'],row['Size_ICV_LUK'],row['CO2_ICV_LUK'],row['Charginginfrastructure_ICV_LUK'],row['Chargingspeed_ICV_LUK'],row['Homedistnearestcharger_ICV_LUK'],row['Homechargeravailability_ICV_LUK']))

    v7 = (row['AV7'], V7(np.random.normal(),row['Purchaseprice_BEV_MINI'],row['Operationcost_BEV_MINI'],row['Yearlycost__BEV_MINI'],row['Range_BEV_MINI'],row['Acceleration_BEV_MINI'],row['Size_BEV_MINI'],row['CO2_BEV_MINI'],row['Charginginfrastructure_BEV_MINI'],row['Chargingspeed_BEV_MINI'],row['Homedistnearestcharger_BEV_MINI'],row['Homechargeravailability_BEV_MINI']))
    v8 = (row['AV8'], V8(np.random.normal(),row['Purchaseprice_BEV_LIL'],row['Operationcost_BEV_LIL'],row['Yearlycost__BEV_LIL'],row['Range_BEV_LIL'],row['Acceleration_BEV_LIL'],row['Size_BEV_LIL'],row['CO2_BEV_LIL'],row['Charginginfrastructure_BEV_LIL'],row['Chargingspeed_BEV_LIL'],row['Homedistnearestcharger_BEV_LIL'],row['Homechargeravailability_BEV_LIL']))
    v9 = (row['AV9'], V9(np.random.normal(),row['Purchaseprice_BEV_MEL'],row['Operationcost_BEV_MEL'],row['Yearlycost__BEV_MEL'],row['Range_BEV_MEL'],row['Acceleration_BEV_MEL'],row['Size_BEV_MEL'],row['CO2_BEV_MEL'],row['Charginginfrastructure_BEV_MEL'],row['Chargingspeed_BEV_MEL'],row['Homedistnearestcharger_BEV_MEL'],row['Homechargeravailability_BEV_MEL']))
    v10 = (row['AV10'], V10(np.random.normal(),row['Purchaseprice_BEV_STOR'],row['Operationcost_BEV_STOR'],row['Yearlycost__BEV_STOR'],row['Range_BEV_STOR'],row['Acceleration_BEV_STOR'],row['Size_BEV_STOR'],row['CO2_BEV_STOR'],row['Charginginfrastructure_BEV_STOR'],row['Chargingspeed_BEV_STOR'],row['Homedistnearestcharger_BEV_STOR'],row['Homechargeravailability_BEV_STOR']))
    v11 = (row['AV11'], V11(np.random.normal(),row['Purchaseprice_BEV_PREM'],row['Operationcost_BEV_PREM'],row['Yearlycost__BEV_PREM'],row['Range_BEV_PREM'],row['Acceleration_BEV_PREM'],row['Size_BEV_PREM'],row['CO2_BEV_PREM'],row['Charginginfrastructure_BEV_PREM'],row['Chargingspeed_BEV_PREM'],row['Homedistnearestcharger_BEV_PREM'],row['Homechargeravailability_BEV_PREM']))
    v12 = (row['AV12'], V12(np.random.normal(),row['Purchaseprice_BEV_LUK'],row['Operationcost_BEV_LUK'],row['Yearlycost__BEV_LUK'],row['Range_BEV_LUK'],row['Acceleration_BEV_LUK'],row['Size_BEV_LUK'],row['CO2_BEV_LUK'],row['Charginginfrastructure_BEV_LUK'],row['Chargingspeed_BEV_LUK'],row['Homedistnearestcharger_BEV_LUK'],row['Homechargeravailability_BEV_LUK']))

    v13 = (row['AV13'], V13(np.random.normal(),row['Purchaseprice_PHEV_MINI'],row['Operationcost_PHEV_MINI'],row['Yearlycost__PHEV_MINI'],row['Range_PHEV_MINI'],row['Acceleration_PHEV_MINI'],row['Size_PHEV_MINI'],row['CO2_PHEV_MINI'],row['Charginginfrastructure_PHEV_MINI'],row['Chargingspeed_PHEV_MINI'],row['Homedistnearestcharger_PHEV_MINI'],row['Homechargeravailability_PHEV_MIN']))
    v14 = (row['AV14'], V14(np.random.normal(),row['Purchaseprice_PHEV_LIL'],row['Operationcost_PHEV_LIL'],row['Yearlycost__PHEV_LIL'],row['Range_PHEV_LIL'],row['Acceleration_PHEV_LIL'],row['Size_PHEV_LIL'],row['CO2_PHEV_LIL'],row['Charginginfrastructure_PHEV_LIL'],row['Chargingspeed_PHEV_LIL'],row['Homedistnearestcharger_PHEV_LIL'],row['Homechargeravailability_PHEV_LIL']))
    v15 = (row['AV15'], V15(np.random.normal(),row['Purchaseprice_PHEV_MEL'],row['Operationcost_PHEV_MEL'],row['Yearlycost__PHEV_MEL'],row['Range_PHEV_MEL'],row['Acceleration_PHEV_MEL'],row['Size_PHEV_MEL'],row['CO2_PHEV_MEL'],row['Charginginfrastructure_PHEV_MEL'],row['Chargingspeed_PHEV_MEL'],row['Homedistnearestcharger_PHEV_MEL'],row['Homechargeravailability_PHEV_MEL']))
    v16 = (row['AV16'], V16(np.random.normal(),row['Purchaseprice_PHEV_STOR'],row['Operationcost_PHEV_STOR'],row['Yearlycost__PHEV_STOR'],row['Range_PHEV_STOR'],row['Acceleration_PHEV_STOR'],row['Size_PHEV_STOR'],row['CO2_PHEV_STOR'],row['Charginginfrastructure_PHEV_STOR'],row['Chargingspeed_PHEV_STOR'],row['Homedistnearestcharger_PHEV_STOR'],row['Homechargeravailability_PHEV_STO']))   
    v17 = (row['AV17'], V17(np.random.normal(),row['Purchaseprice_PHEV_PREM'],row['Operationcost_PHEV_PREM'],row['Yearlycost__PHEV_PREM'],row['Range_PHEV_PREM'],row['Acceleration_PHEV_PREM'],row['Size_PHEV_PREM'],row['CO2_PHEV_PREM'],row['Charginginfrastructure_PHEV_PREM'],row['Chargingspeed_PHEV_PREM'],row['Homedistnearestcharger_PHEV_PREM'],row['Homechargeravailability_PHEV_PRE']))
    v18 = (row['AV18'], V18(row['Purchaseprice_PHEV_LUK'],row['Operationcost_PHEV_LUK'],row['Yearlycost__PHEV_LUK'],row['Range_PHEV_LUK'],row['Acceleration_PHEV_LUK'],row['Size_PHEV_LUK'],row['CO2_PHEV_LUK'],row['Charginginfrastructure_PHEV_LUK'],row['Chargingspeed_PHEV_LUK'],row['Homedistnearestcharger_PHEV_LUK'],row['Homechargeravailability_PHEV_LUK']))
    
    
    probVector = prob(mu, v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18)
    probMatrix.append(probVector)

Sample enumeration

In [87]:
tot = len(probMatrix)
alt = len(mapping)
sampleEnum = {}
for i in range(len(mapping)):
    sampleEnum[mapping[i]] = round((sum([x[i] for x in probMatrix]) / tot) * 100,3)
df_res = pd.DataFrame([sampleEnum])
df_res.style.hide_index()

ICEV_1,ICEV_2,ICEV_3,ICEV_4,ICEV_5,ICEV_6,BEV_1,BEV_2,BEV_3,BEV_4,BEV_5,BEV_6,PHEV_1,PHEV_2,PHEV_3,PHEV_4,PHEV_5,PHEV_6
6.223000,9.271000,15.207000,6.768000,3.549000,0.779000,5.748000,6.285000,16.088000,9.498000,3.642000,0.679000,2.104000,2.748000,7.130000,3.153000,0.831000,0.297000


Simulation

In [88]:
r = []
for i in range(1000):
    x = [sim(x) for x in probMatrix]
    r.append({mapping[x[0]]:round((x[1]/ n)*100,3) for x in list(zip(Counter(x).keys(), Counter(x).values()))})
    
R = pd.DataFrame(r)

In [89]:
R.transpose()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,...,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
BEV_2,6.281,6.281,6.260,6.374,6.171,6.243,6.437,6.142,6.370,6.319,6.247,6.188,6.273,6.230,6.454,6.391,6.108,6.205,6.264,6.425,6.171,6.294,6.163,6.421,6.391,...,6.437,6.235,6.484,6.302,6.214,6.328,6.336,6.230,6.201,6.370,6.328,6.201,6.268,6.176,6.040,6.391,6.243,6.268,6.273,6.306,6.146,6.188,6.247,6.378,6.340
ICEV_2,9.204,9.154,9.259,9.398,9.251,9.204,9.242,9.200,9.238,9.272,9.251,9.331,9.170,9.238,9.466,9.263,9.276,9.187,9.348,9.154,9.246,9.217,9.411,9.225,8.997,...,9.128,9.356,9.234,9.120,9.137,9.437,9.255,9.280,9.403,9.331,9.263,9.276,9.238,9.348,9.365,9.149,9.424,9.170,9.103,9.280,9.403,9.297,9.272,9.225,9.331
PHEV_2,2.674,2.682,2.712,2.784,2.767,2.678,2.898,2.826,2.678,2.771,2.784,2.737,2.670,2.813,2.699,2.792,2.716,2.754,2.796,2.775,2.923,2.750,2.657,2.716,2.805,...,2.670,2.644,2.746,2.746,2.767,2.670,2.712,2.801,2.670,2.801,2.864,2.720,2.809,2.792,2.712,2.813,2.729,2.792,2.864,2.784,2.737,2.725,2.653,2.733,2.750
ICEV_4,6.564,6.661,6.970,6.784,6.864,6.746,6.771,6.737,6.873,6.898,6.906,6.771,6.915,6.775,6.767,7.016,6.644,6.780,6.885,6.560,6.687,6.788,6.788,6.687,6.649,...,6.868,6.835,6.813,6.725,6.822,6.699,7.020,6.636,6.611,6.742,6.911,6.720,6.750,6.775,6.535,6.758,6.644,6.889,6.704,6.640,6.754,6.733,6.826,6.839,6.699
BEV_3,16.030,16.187,16.047,15.992,16.372,16.013,15.942,16.039,16.284,16.127,16.051,16.089,15.967,16.140,16.051,16.022,16.343,15.925,16.157,16.161,16.149,16.127,16.081,16.051,16.157,...,16.111,16.026,16.220,15.912,15.996,16.140,15.874,16.199,16.279,16.275,15.798,16.018,16.051,16.034,16.115,16.001,16.111,16.199,16.043,16.187,16.034,16.212,15.975,15.840,16.385
BEV_4,9.517,9.369,9.453,9.390,9.618,9.487,9.428,9.770,9.398,9.508,9.546,9.327,9.348,9.360,9.496,9.268,9.466,9.580,9.513,9.437,9.496,9.373,9.525,9.698,9.504,...,9.483,9.466,9.386,9.517,9.610,9.335,9.420,9.698,9.297,9.453,9.428,9.390,9.529,9.652,9.407,9.369,9.259,9.677,9.686,9.500,9.669,9.382,9.567,9.432,9.715
ICEV_3,15.359,15.232,15.021,15.084,15.156,15.544,15.287,15.240,15.181,15.240,15.067,15.435,15.173,15.207,15.211,15.333,15.109,15.515,15.076,15.093,15.198,15.211,15.435,14.924,15.304,...,15.088,15.245,15.228,15.232,15.169,15.333,14.974,15.278,15.477,14.902,15.329,15.426,14.949,15.054,15.464,15.359,15.477,14.936,15.329,15.135,15.088,15.278,15.131,14.936,14.818
PHEV_3,7.156,7.194,7.113,7.139,7.020,7.101,6.911,7.088,7.096,7.096,7.151,7.168,7.122,7.105,7.134,7.063,7.223,7.025,6.750,7.232,7.206,7.219,6.957,7.084,7.156,...,7.118,7.160,7.037,7.329,6.999,7.058,7.118,7.063,7.067,7.012,7.215,7.202,7.143,7.122,7.185,7.084,7.210,7.084,7.003,7.219,7.075,7.084,7.215,7.346,7.143
PHEV_4,3.291,3.109,3.168,3.345,3.003,3.075,3.291,3.206,3.122,3.151,3.138,3.244,3.210,3.037,3.109,3.109,3.181,3.024,3.151,3.193,3.214,3.062,3.084,3.088,3.160,...,3.244,3.168,3.062,3.214,3.092,3.227,3.143,2.961,3.122,3.261,2.965,3.092,3.147,3.227,3.202,3.155,3.185,3.143,3.236,3.160,3.084,3.168,3.172,3.143,3.062
ICEV_1,6.150,6.306,6.311,6.184,6.188,6.277,6.319,6.091,6.108,6.180,6.214,6.049,6.501,6.239,6.100,6.095,6.176,6.387,6.323,6.268,6.083,6.294,6.230,6.091,6.260,...,6.108,6.268,6.062,6.201,6.349,6.197,6.378,6.336,6.205,6.180,6.066,6.108,6.353,6.214,6.273,6.116,6.247,6.315,6.222,6.243,6.353,6.214,6.281,6.353,6.226


In [90]:
R.describe()

,BEV_2,ICEV_2,PHEV_2,ICEV_4,BEV_3,BEV_4,ICEV_3,PHEV_3,PHEV_4,ICEV_1,BEV_1,ICEV_5,BEV_5,PHEV_1,ICEV_6,PHEV_5,BEV_6,PHEV_6
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,6.287093,9.269748,2.754363,6.768992,16.089442,9.492776,15.208354,7.129667,3.153273,6.223237,5.742962,3.546645,3.642826,2.104724,0.776839,0.832854,0.679134,0.297042
std,0.105807,0.112445,0.070704,0.107268,0.152668,0.127996,0.142839,0.108095,0.078370,0.090915,0.099275,0.074346,0.073677,0.063605,0.032166,0.042386,0.037832,0.027916
min,5.943000,8.925000,2.543000,6.425000,15.502000,9.137000,14.797000,6.750000,2.915000,5.922000,5.403000,3.299000,3.438000,1.875000,0.676000,0.714000,0.541000,0.203000
25%,6.218000,9.196000,2.703000,6.699000,15.988000,9.398000,15.109000,7.054000,3.096000,6.163000,5.677000,3.493000,3.590000,2.066000,0.756000,0.803000,0.655000,0.279000
50%,6.285000,9.268000,2.754000,6.767000,16.089000,9.487000,15.207000,7.130000,3.151000,6.222000,5.745000,3.548000,3.641000,2.104000,0.773000,0.832000,0.680000,0.296000
75%,6.358000,9.348000,2.805000,6.847000,16.191000,9.576000,15.305000,7.202000,3.206000,6.281000,5.804000,3.595000,3.692000,2.150000,0.798000,0.862000,0.701000,0.317000
max,6.606000,9.639000,2.978000,7.147000,16.605000,9.981000,15.616000,7.464000,3.405000,6.501000,6.159000,3.776000,3.878000,2.289000,0.900000,0.946000,0.803000,0.384000


Observed market shares

In [91]:
df_test = df_mod.copy(deep=True)
df_test['count'] = 0
df_test = df_test.groupby('CHOICE18', as_index=False)['count'].count()
df_test.transpose()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
CHOICE18,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
count,1538,2184,3604,1574,864,187,1254,1583,4118,2177,660,137,543,670,1713,670,150,48


In [92]:
df_test['share'] = df_test['count'] / n * 100
df_test = df_test[['CHOICE18','share']].transpose()
df_test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
CHOICE18,1.000000,2.00000,3.000000,4.000000,5.000000,6.000000,7.00000,8.00000,9.00000,10.000000,11.000000,12.000000,13.000000,14.000000,15.000000,16.000000,17.000000,18.000000
share,6.496579,9.22531,15.223452,6.648644,3.649573,0.789896,5.29695,6.68666,17.39461,9.195742,2.787869,0.578694,2.293655,2.830109,7.235786,2.830109,0.633606,0.202754


#### Scenario 1
Change in cost and range of BEV/PHEV (better technologies, scaling-up production, subsidies).

In [93]:
df_s1 = df_mod.copy(deep=True)

In [94]:
col = df_s1.columns.to_list()
col_bev = []
col_phev = []
for c in col:
    if Search("Purchaseprice_BEV_",c):
        col_bev.append(c)
    if Search("Purchaseprice_PHEV_",c):
        col_phev.append(c)
print(col_bev)
print(col_phev)

['Purchaseprice_BEV_MINI', 'Purchaseprice_BEV_LIL', 'Purchaseprice_BEV_MEL', 'Purchaseprice_BEV_STOR', 'Purchaseprice_BEV_PREM', 'Purchaseprice_BEV_LUK']
['Purchaseprice_PHEV_MINI', 'Purchaseprice_PHEV_LIL', 'Purchaseprice_PHEV_MEL', 'Purchaseprice_PHEV_STOR', 'Purchaseprice_PHEV_PREM', 'Purchaseprice_PHEV_LUK']


In [95]:
for c in col_bev:
    df_s1[c+"_NEW"] = df_s1[c] * 0.6

df_s1[df_s1['Purchaseprice_BEV_MINI'] != 0][['Purchaseprice_BEV_MINI','Purchaseprice_BEV_MINI_NEW']].head(2)

,Purchaseprice_BEV_MINI,Purchaseprice_BEV_MINI_NEW
24,125000,75000.0
25,125000,75000.0


In [96]:
for c in col_phev:
    df_s1[c+"_NEW"] = df_s1[c] * 0.8

df_s1[df_s1['Purchaseprice_PHEV_MINI'] != 0][['Purchaseprice_PHEV_MINI','Purchaseprice_PHEV_MINI_NEW']].head(2)

,Purchaseprice_PHEV_MINI,Purchaseprice_PHEV_MINI_NEW
28,135000,108000.0
29,135000,108000.0


In [97]:
col = df_s1.columns.to_list()
col_bev_r = []
col_phev_r = []
for c in col:
    if Search("Range_BEV_",c):
        col_bev_r.append(c)
    if Search("Range_PHEV_",c):
        col_phev_r.append(c)
print(col_bev_r)
print(col_phev_r)

['Range_BEV_MINI', 'Range_BEV_LIL', 'Range_BEV_MEL', 'Range_BEV_STOR', 'Range_BEV_PREM', 'Range_BEV_LUK']
['Range_PHEV_MINI', 'Range_PHEV_LIL', 'Range_PHEV_MEL', 'Range_PHEV_STOR', 'Range_PHEV_PREM', 'Range_PHEV_LUK']


In [101]:
for c in col_phev_r:
    df_s1[c+"_NEW"] = df_s1[c] * 2

#df_s1[df_s1['Range_BEV_MINI'] != 0][['Range_BEV_MINI','Range_BEV_MINI_NEW']].head(2)

In [102]:
for c in col_bev_r:
    df_s1[c+"_NEW"] = df_s1[c] * 1.5

df_s1[df_s1['Range_PHEV_MINI'] != 0][['Range_PHEV_MINI','Range_PHEV_MINI_NEW']].head(2)

,Range_PHEV_MINI,Range_PHEV_MINI_NEW
28,60,120
29,60,120


Calculating probabilities

In [104]:
mu = 1

probMatrix_s1 = []
for index, row in df_s1.iterrows():
    v1 = (row['AV1'], V1(np.random.normal(),row['Purchaseprice_ICV_MINI'],row['Operationcost_ICV_MINI'],row['Yearlycost__ICV_MINI'],row['Range_ICV_MINI'],row['Acceleration_ICV_MINI'],row['Size_ICV1'],row['CO2_ICV_MINI'],row['Charginginfrastructure_ICV_MINI'],row['Chargingspeed_ICV_MINI'],row['Homedistnearestcharger_ICV_MINI'],row['Homechargeravailability_ICV_MINI']))
    v2 = (row['AV2'], V2(np.random.normal(),row['Purchaseprice_ICV_LIL'],row['Operationcost_ICV_LIL'],row['Yearlycost__ICV_LIL'],row['Range_ICV_LIL'],row['Acceleration_ICV_LIL'],row['Size_ICV_LIL'],row['CO2_ICV_LIL'],row['Charginginfrastructure_ICV_LIL'],row['Chargingspeed_ICV_LIL'],row['Homedistnearestcharger_ICV_LIL'],row['Homechargeravailability_ICV_LIL']))
    v3 = (row['AV3'], V3(np.random.normal(),row['Purchaseprice_ICV_MEL'],row['Operationcost_ICV_MEL'],row['Yearlycost__ICV_MEL'],row['Range_ICV_MEL'],row['Acceleration_ICV_MEL'],row['Size_ICV_MEL'],row['CO2_ICV_MEL'],row['Charginginfrastructure_ICV_MEL'],row['Chargingspeed_ICV_MEL'],row['Homedistnearestcharger_ICV_MEL'],row['Homechargeravailability_ICV_MEL']))
    v4 = (row['AV4'], V4(np.random.normal(),row['Purchaseprice_ICV_STOR'],row['Operationcost_ICV_STOR'],row['Yearlycost__ICV_STOR'],row['Range_ICV_STOR'],row['Acceleration_ICV_STOR'],row['Size_ICV_STOR'],row['CO2_ICV_STOR'],row['Charginginfrastructure_ICV_STOR'],row['Chargingspeed_ICV_STOR'],row['Homedistnearestcharger_ICV_STOR'],row['Homechargeravailability_ICV_STOR']))
    v5 = (row['AV5'], V5(np.random.normal(),row['Purchaseprice_ICV_PREM'],row['Operationcost_ICV_PREM'],row['Yearlycost__ICV_PREM'],row['Range_ICV_PREM'],row['Acceleration_ICV_PREM'],row['Size_ICV_PREM'],row['CO2_ICV_PREM'],row['Charginginfrastructure_ICV_PREM'],row['Chargingspeed_ICV_PREM'],row['Homedistnearestcharger_ICV_PREM'],row['Homechargeravailability_ICV_PREM']))
    v6 = (row['AV6'], V6(np.random.normal(),row['Purchaseprice_ICV_LUK'],row['Operationcost_ICV_LUK'],row['Yearlycost__ICV_LUK'],row['Range_ICV_LUK'],row['Acceleration_ICV_LUK'],row['Size_ICV_LUK'],row['CO2_ICV_LUK'],row['Charginginfrastructure_ICV_LUK'],row['Chargingspeed_ICV_LUK'],row['Homedistnearestcharger_ICV_LUK'],row['Homechargeravailability_ICV_LUK']))

    v7 = (row['AV7'], V7(np.random.normal(),row['Purchaseprice_BEV_MINI_NEW'],row['Operationcost_BEV_MINI'],row['Yearlycost__BEV_MINI'],row['Range_BEV_MINI_NEW'],row['Acceleration_BEV_MINI'],row['Size_BEV_MINI'],row['CO2_BEV_MINI'],row['Charginginfrastructure_BEV_MINI'],row['Chargingspeed_BEV_MINI'],row['Homedistnearestcharger_BEV_MINI'],row['Homechargeravailability_BEV_MINI']))
    v8 = (row['AV8'], V8(np.random.normal(),row['Purchaseprice_BEV_LIL_NEW'],row['Operationcost_BEV_LIL'],row['Yearlycost__BEV_LIL'],row['Range_BEV_LIL_NEW'],row['Acceleration_BEV_LIL'],row['Size_BEV_LIL'],row['CO2_BEV_LIL'],row['Charginginfrastructure_BEV_LIL'],row['Chargingspeed_BEV_LIL'],row['Homedistnearestcharger_BEV_LIL'],row['Homechargeravailability_BEV_LIL']))
    v9 = (row['AV9'], V9(np.random.normal(),row['Purchaseprice_BEV_MEL_NEW'],row['Operationcost_BEV_MEL'],row['Yearlycost__BEV_MEL'],row['Range_BEV_MEL_NEW'],row['Acceleration_BEV_MEL'],row['Size_BEV_MEL'],row['CO2_BEV_MEL'],row['Charginginfrastructure_BEV_MEL'],row['Chargingspeed_BEV_MEL'],row['Homedistnearestcharger_BEV_MEL'],row['Homechargeravailability_BEV_MEL']))
    v10 = (row['AV10'], V10(np.random.normal(),row['Purchaseprice_BEV_STOR_NEW'],row['Operationcost_BEV_STOR'],row['Yearlycost__BEV_STOR'],row['Range_BEV_STOR_NEW'],row['Acceleration_BEV_STOR'],row['Size_BEV_STOR'],row['CO2_BEV_STOR'],row['Charginginfrastructure_BEV_STOR'],row['Chargingspeed_BEV_STOR'],row['Homedistnearestcharger_BEV_STOR'],row['Homechargeravailability_BEV_STOR']))
    v11 = (row['AV11'], V11(np.random.normal(),row['Purchaseprice_BEV_PREM_NEW'],row['Operationcost_BEV_PREM'],row['Yearlycost__BEV_PREM'],row['Range_BEV_PREM_NEW'],row['Acceleration_BEV_PREM'],row['Size_BEV_PREM'],row['CO2_BEV_PREM'],row['Charginginfrastructure_BEV_PREM'],row['Chargingspeed_BEV_PREM'],row['Homedistnearestcharger_BEV_PREM'],row['Homechargeravailability_BEV_PREM']))
    v12 = (row['AV12'], V12(np.random.normal(),row['Purchaseprice_BEV_LUK_NEW'],row['Operationcost_BEV_LUK'],row['Yearlycost__BEV_LUK'],row['Range_BEV_LUK_NEW'],row['Acceleration_BEV_LUK'],row['Size_BEV_LUK'],row['CO2_BEV_LUK'],row['Charginginfrastructure_BEV_LUK'],row['Chargingspeed_BEV_LUK'],row['Homedistnearestcharger_BEV_LUK'],row['Homechargeravailability_BEV_LUK']))

    v13 = (row['AV13'], V13(np.random.normal(),row['Purchaseprice_PHEV_MINI_NEW'],row['Operationcost_PHEV_MINI'],row['Yearlycost__PHEV_MINI'],row['Range_PHEV_MINI_NEW'],row['Acceleration_PHEV_MINI'],row['Size_PHEV_MINI'],row['CO2_PHEV_MINI'],row['Charginginfrastructure_PHEV_MINI'],row['Chargingspeed_PHEV_MINI'],row['Homedistnearestcharger_PHEV_MINI'],row['Homechargeravailability_PHEV_MIN']))
    v14 = (row['AV14'], V14(np.random.normal(),row['Purchaseprice_PHEV_LIL_NEW'],row['Operationcost_PHEV_LIL'],row['Yearlycost__PHEV_LIL'],row['Range_PHEV_LIL_NEW'],row['Acceleration_PHEV_LIL'],row['Size_PHEV_LIL'],row['CO2_PHEV_LIL'],row['Charginginfrastructure_PHEV_LIL'],row['Chargingspeed_PHEV_LIL'],row['Homedistnearestcharger_PHEV_LIL'],row['Homechargeravailability_PHEV_LIL']))
    v15 = (row['AV15'], V15(np.random.normal(),row['Purchaseprice_PHEV_MEL_NEW'],row['Operationcost_PHEV_MEL'],row['Yearlycost__PHEV_MEL'],row['Range_PHEV_MEL_NEW'],row['Acceleration_PHEV_MEL'],row['Size_PHEV_MEL'],row['CO2_PHEV_MEL'],row['Charginginfrastructure_PHEV_MEL'],row['Chargingspeed_PHEV_MEL'],row['Homedistnearestcharger_PHEV_MEL'],row['Homechargeravailability_PHEV_MEL']))
    v16 = (row['AV16'], V16(np.random.normal(),row['Purchaseprice_PHEV_STOR_NEW'],row['Operationcost_PHEV_STOR'],row['Yearlycost__PHEV_STOR'],row['Range_PHEV_STOR_NEW'],row['Acceleration_PHEV_STOR'],row['Size_PHEV_STOR'],row['CO2_PHEV_STOR'],row['Charginginfrastructure_PHEV_STOR'],row['Chargingspeed_PHEV_STOR'],row['Homedistnearestcharger_PHEV_STOR'],row['Homechargeravailability_PHEV_STO']))    
    v17 = (row['AV17'], V17(np.random.normal(),row['Purchaseprice_PHEV_PREM_NEW'],row['Operationcost_PHEV_PREM'],row['Yearlycost__PHEV_PREM'],row['Range_PHEV_PREM_NEW'],row['Acceleration_PHEV_PREM'],row['Size_PHEV_PREM'],row['CO2_PHEV_PREM'],row['Charginginfrastructure_PHEV_PREM'],row['Chargingspeed_PHEV_PREM'],row['Homedistnearestcharger_PHEV_PREM'],row['Homechargeravailability_PHEV_PRE']))
    v18 = (row['AV18'], V18(row['Purchaseprice_PHEV_LUK_NEW'],row['Operationcost_PHEV_LUK'],row['Yearlycost__PHEV_LUK'],row['Range_PHEV_LUK_NEW'],row['Acceleration_PHEV_LUK'],row['Size_PHEV_LUK'],row['CO2_PHEV_LUK'],row['Charginginfrastructure_PHEV_LUK'],row['Chargingspeed_PHEV_LUK'],row['Homedistnearestcharger_PHEV_LUK'],row['Homechargeravailability_PHEV_LUK']))    
    
    
    probVector_s1 = prob(mu, v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18)
    probMatrix_s1.append(probVector_s1)

Sample enumeration

In [105]:
tot = len(probMatrix_s1)
alt = len(mapping)
sampleEnum_s1 = {}
for i in range(len(mapping)):
    sampleEnum_s1[mapping[i]] = round((sum([x[i] for x in probMatrix_s1]) / tot) * 100,3)
df_res_s1 = pd.DataFrame([sampleEnum_s1])
df_res_s1.style.hide_index()

ICEV_1,ICEV_2,ICEV_3,ICEV_4,ICEV_5,ICEV_6,BEV_1,BEV_2,BEV_3,BEV_4,BEV_5,BEV_6,PHEV_1,PHEV_2,PHEV_3,PHEV_4,PHEV_5,PHEV_6
5.632000,7.479000,11.265000,4.393000,2.142000,0.500000,5.800000,7.406000,19.543000,12.905000,5.301000,2.284000,1.975000,2.502000,6.818000,2.877000,0.815000,0.362000


#### Scenario 2
Better availability of charging stations (investments in infrastructure, subsidies).

In [106]:
df_s2 = df_mod.copy(deep=True)

In [107]:
col = df_s2.columns.to_list()
col_speed = []
col_dist = []
for c in col:
    if Search("Chargingspeed_...*_",c):
        col_speed.append(c)
    if Search("Homedistnearestcharger_...*_",c):
        col_dist.append(c)
#print(col_speed)
#print(col_dist)

In [108]:
for c in col_speed:
    df_s2[c+"_NEW"] = df_s2[c] * 0.5

df_s2[df_s2['Chargingspeed_BEV_MINI'] != 0][['Chargingspeed_BEV_MINI','Chargingspeed_BEV_MINI_NEW']].head(2)

,Chargingspeed_BEV_MINI,Chargingspeed_BEV_MINI_NEW
24,150,75.0
25,75,37.5


In [109]:
for c in col_dist:
    df_s2[c+"_NEW"] = df_s2[c] * 0.5

df_s2[df_s2['Homedistnearestcharger_BEV_MINI'] != 0][['Homedistnearestcharger_BEV_MINI','Homedistnearestcharger_BEV_MINI_NEW']].head(2)

,Homedistnearestcharger_BEV_MINI,Homedistnearestcharger_BEV_MINI_NEW
24,600,300.0
25,50,25.0


Calculating probabilities

In [110]:
mu = 1

probMatrix_s2 = []
for index, row in df_s2.iterrows():
    v1 = (row['AV1'], V1(np.random.normal(),row['Purchaseprice_ICV_MINI'],row['Operationcost_ICV_MINI'],row['Yearlycost__ICV_MINI'],row['Range_ICV_MINI'],row['Acceleration_ICV_MINI'],row['Size_ICV1'],row['CO2_ICV_MINI'],row['Charginginfrastructure_ICV_MINI'],row['Chargingspeed_ICV_MINI'],row['Homedistnearestcharger_ICV_MINI'],row['Homechargeravailability_ICV_MINI']))
    v2 = (row['AV2'], V2(np.random.normal(),row['Purchaseprice_ICV_LIL'],row['Operationcost_ICV_LIL'],row['Yearlycost__ICV_LIL'],row['Range_ICV_LIL'],row['Acceleration_ICV_LIL'],row['Size_ICV_LIL'],row['CO2_ICV_LIL'],row['Charginginfrastructure_ICV_LIL'],row['Chargingspeed_ICV_LIL'],row['Homedistnearestcharger_ICV_LIL'],row['Homechargeravailability_ICV_LIL']))
    v3 = (row['AV3'], V3(np.random.normal(),row['Purchaseprice_ICV_MEL'],row['Operationcost_ICV_MEL'],row['Yearlycost__ICV_MEL'],row['Range_ICV_MEL'],row['Acceleration_ICV_MEL'],row['Size_ICV_MEL'],row['CO2_ICV_MEL'],row['Charginginfrastructure_ICV_MEL'],row['Chargingspeed_ICV_MEL'],row['Homedistnearestcharger_ICV_MEL'],row['Homechargeravailability_ICV_MEL']))
    v4 = (row['AV4'], V4(np.random.normal(),row['Purchaseprice_ICV_STOR'],row['Operationcost_ICV_STOR'],row['Yearlycost__ICV_STOR'],row['Range_ICV_STOR'],row['Acceleration_ICV_STOR'],row['Size_ICV_STOR'],row['CO2_ICV_STOR'],row['Charginginfrastructure_ICV_STOR'],row['Chargingspeed_ICV_STOR'],row['Homedistnearestcharger_ICV_STOR'],row['Homechargeravailability_ICV_STOR']))
    v5 = (row['AV5'], V5(np.random.normal(),row['Purchaseprice_ICV_PREM'],row['Operationcost_ICV_PREM'],row['Yearlycost__ICV_PREM'],row['Range_ICV_PREM'],row['Acceleration_ICV_PREM'],row['Size_ICV_PREM'],row['CO2_ICV_PREM'],row['Charginginfrastructure_ICV_PREM'],row['Chargingspeed_ICV_PREM'],row['Homedistnearestcharger_ICV_PREM'],row['Homechargeravailability_ICV_PREM']))
    v6 = (row['AV6'], V6(np.random.normal(),row['Purchaseprice_ICV_LUK'],row['Operationcost_ICV_LUK'],row['Yearlycost__ICV_LUK'],row['Range_ICV_LUK'],row['Acceleration_ICV_LUK'],row['Size_ICV_LUK'],row['CO2_ICV_LUK'],row['Charginginfrastructure_ICV_LUK'],row['Chargingspeed_ICV_LUK'],row['Homedistnearestcharger_ICV_LUK'],row['Homechargeravailability_ICV_LUK']))

    v7 = (row['AV7'], V7(np.random.normal(),row['Purchaseprice_BEV_MINI'],row['Operationcost_BEV_MINI'],row['Yearlycost__BEV_MINI'],row['Range_BEV_MINI'],row['Acceleration_BEV_MINI'],row['Size_BEV_MINI'],row['CO2_BEV_MINI'],row['Charginginfrastructure_BEV_MINI'],row['Chargingspeed_BEV_MINI_NEW'],row['Homedistnearestcharger_BEV_MINI_NEW'],row['Homechargeravailability_BEV_MINI']))
    v8 = (row['AV8'], V8(np.random.normal(),row['Purchaseprice_BEV_LIL'],row['Operationcost_BEV_LIL'],row['Yearlycost__BEV_LIL'],row['Range_BEV_LIL'],row['Acceleration_BEV_LIL'],row['Size_BEV_LIL'],row['CO2_BEV_LIL'],row['Charginginfrastructure_BEV_LIL'],row['Chargingspeed_BEV_LIL_NEW'],row['Homedistnearestcharger_BEV_LIL_NEW'],row['Homechargeravailability_BEV_LIL']))
    v9 = (row['AV9'], V9(np.random.normal(),row['Purchaseprice_BEV_MEL'],row['Operationcost_BEV_MEL'],row['Yearlycost__BEV_MEL'],row['Range_BEV_MEL'],row['Acceleration_BEV_MEL'],row['Size_BEV_MEL'],row['CO2_BEV_MEL'],row['Charginginfrastructure_BEV_MEL'],row['Chargingspeed_BEV_MEL_NEW'],row['Homedistnearestcharger_BEV_MEL_NEW'],row['Homechargeravailability_BEV_MEL']))
    v10 = (row['AV10'], V10(np.random.normal(),row['Purchaseprice_BEV_STOR'],row['Operationcost_BEV_STOR'],row['Yearlycost__BEV_STOR'],row['Range_BEV_STOR'],row['Acceleration_BEV_STOR'],row['Size_BEV_STOR'],row['CO2_BEV_STOR'],row['Charginginfrastructure_BEV_STOR'],row['Chargingspeed_BEV_STOR_NEW'],row['Homedistnearestcharger_BEV_STOR_NEW'],row['Homechargeravailability_BEV_STOR']))
    v11 = (row['AV11'], V11(np.random.normal(),row['Purchaseprice_BEV_PREM'],row['Operationcost_BEV_PREM'],row['Yearlycost__BEV_PREM'],row['Range_BEV_PREM'],row['Acceleration_BEV_PREM'],row['Size_BEV_PREM'],row['CO2_BEV_PREM'],row['Charginginfrastructure_BEV_PREM'],row['Chargingspeed_BEV_PREM_NEW'],row['Homedistnearestcharger_BEV_PREM_NEW'],row['Homechargeravailability_BEV_PREM']))
    v12 = (row['AV12'], V12(np.random.normal(),row['Purchaseprice_BEV_LUK'],row['Operationcost_BEV_LUK'],row['Yearlycost__BEV_LUK'],row['Range_BEV_LUK'],row['Acceleration_BEV_LUK'],row['Size_BEV_LUK'],row['CO2_BEV_LUK'],row['Charginginfrastructure_BEV_LUK'],row['Chargingspeed_BEV_LUK_NEW'],row['Homedistnearestcharger_BEV_LUK_NEW'],row['Homechargeravailability_BEV_LUK']))

    v13 = (row['AV13'], V13(np.random.normal(),row['Purchaseprice_PHEV_MINI'],row['Operationcost_PHEV_MINI'],row['Yearlycost__PHEV_MINI'],row['Range_PHEV_MINI'],row['Acceleration_PHEV_MINI'],row['Size_PHEV_MINI'],row['CO2_PHEV_MINI'],row['Charginginfrastructure_PHEV_MINI'],row['Chargingspeed_PHEV_MINI_NEW'],row['Homedistnearestcharger_PHEV_MINI_NEW'],row['Homechargeravailability_PHEV_MIN']))
    v14 = (row['AV14'], V14(np.random.normal(),row['Purchaseprice_PHEV_LIL'],row['Operationcost_PHEV_LIL'],row['Yearlycost__PHEV_LIL'],row['Range_PHEV_LIL'],row['Acceleration_PHEV_LIL'],row['Size_PHEV_LIL'],row['CO2_PHEV_LIL'],row['Charginginfrastructure_PHEV_LIL'],row['Chargingspeed_PHEV_LIL_NEW'],row['Homedistnearestcharger_PHEV_LIL_NEW'],row['Homechargeravailability_PHEV_LIL']))
    v15 = (row['AV15'], V15(np.random.normal(),row['Purchaseprice_PHEV_MEL'],row['Operationcost_PHEV_MEL'],row['Yearlycost__PHEV_MEL'],row['Range_PHEV_MEL'],row['Acceleration_PHEV_MEL'],row['Size_PHEV_MEL'],row['CO2_PHEV_MEL'],row['Charginginfrastructure_PHEV_MEL'],row['Chargingspeed_PHEV_MEL_NEW'],row['Homedistnearestcharger_PHEV_MEL_NEW'],row['Homechargeravailability_PHEV_MEL']))
    v16 = (row['AV16'], V16(np.random.normal(),row['Purchaseprice_PHEV_STOR'],row['Operationcost_PHEV_STOR'],row['Yearlycost__PHEV_STOR'],row['Range_PHEV_STOR'],row['Acceleration_PHEV_STOR'],row['Size_PHEV_STOR'],row['CO2_PHEV_STOR'],row['Charginginfrastructure_PHEV_STOR'],row['Chargingspeed_PHEV_STOR_NEW'],row['Homedistnearestcharger_PHEV_STOR_NEW'],row['Homechargeravailability_PHEV_STO']))    
    v17 = (row['AV17'], V17(np.random.normal(),row['Purchaseprice_PHEV_PREM'],row['Operationcost_PHEV_PREM'],row['Yearlycost__PHEV_PREM'],row['Range_PHEV_PREM'],row['Acceleration_PHEV_PREM'],row['Size_PHEV_PREM'],row['CO2_PHEV_PREM'],row['Charginginfrastructure_PHEV_PREM'],row['Chargingspeed_PHEV_PREM_NEW'],row['Homedistnearestcharger_PHEV_PREM_NEW'],row['Homechargeravailability_PHEV_PRE']))
    v18 = (row['AV18'], V18(row['Purchaseprice_PHEV_LUK'],row['Operationcost_PHEV_LUK'],row['Yearlycost__PHEV_LUK'],row['Range_PHEV_LUK'],row['Acceleration_PHEV_LUK'],row['Size_PHEV_LUK'],row['CO2_PHEV_LUK'],row['Charginginfrastructure_PHEV_LUK'],row['Chargingspeed_PHEV_LUK_NEW'],row['Homedistnearestcharger_PHEV_LUK_NEW'],row['Homechargeravailability_PHEV_LUK']))
    
    
    probVector_s2 = prob(mu, v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18)
    probMatrix_s2.append(probVector_s2)

Sample enumeration

In [111]:
tot = len(probMatrix_s2)
alt = len(mapping)
sampleEnum_s2 = {}
for i in range(len(mapping)):
    sampleEnum_s2[mapping[i]] = round((sum([x[i] for x in probMatrix_s2]) / tot) * 100,3)
df_res_s2 = pd.DataFrame([sampleEnum_s2])
df_res_s2.style.hide_index()

ICEV_1,ICEV_2,ICEV_3,ICEV_4,ICEV_5,ICEV_6,BEV_1,BEV_2,BEV_3,BEV_4,BEV_5,BEV_6,PHEV_1,PHEV_2,PHEV_3,PHEV_4,PHEV_5,PHEV_6
6.344000,9.306000,15.288000,6.725000,3.661000,0.899000,5.661000,6.134000,15.539000,9.114000,3.492000,0.692000,2.239000,2.838000,7.686000,3.122000,0.969000,0.291000


#### Scenario 3
Higher electricity prices (EU’s climate policy.: electrification, bigger share of intermittent sources, etc.) 

In [112]:
df_s3 = df_mod.copy(deep=True)

In [113]:
col = df_s3.columns.to_list()
col_oc = []
for c in col:
    #if Search("Operationcost_[^I]...*_",c):
    if Search("Operationcost_...*_",c):
        col_oc.append(c)
#col_oc

In [114]:
for c in col_oc:
    df_s3[c+"_NEW"] = df_s2[c] * 1.75

df_s3[df_s3['Operationcost_BEV_MINI'] != 0][['Operationcost_BEV_MINI','Operationcost_BEV_MINI_NEW']].head(2)

,Operationcost_BEV_MINI,Operationcost_BEV_MINI_NEW
24,0.67,1.1725
25,0.49,0.8575


Calculating probabilities

In [115]:
mu = 1

probMatrix_s3 = []
for index, row in df_s3.iterrows():
    v1 = (row['AV1'], V1(np.random.normal(),row['Purchaseprice_ICV_MINI'],row['Operationcost_ICV_MINI'],row['Yearlycost__ICV_MINI'],row['Range_ICV_MINI'],row['Acceleration_ICV_MINI'],row['Size_ICV1'],row['CO2_ICV_MINI'],row['Charginginfrastructure_ICV_MINI'],row['Chargingspeed_ICV_MINI'],row['Homedistnearestcharger_ICV_MINI'],row['Homechargeravailability_ICV_MINI']))
    v2 = (row['AV2'], V2(np.random.normal(),row['Purchaseprice_ICV_LIL'],row['Operationcost_ICV_LIL'],row['Yearlycost__ICV_LIL'],row['Range_ICV_LIL'],row['Acceleration_ICV_LIL'],row['Size_ICV_LIL'],row['CO2_ICV_LIL'],row['Charginginfrastructure_ICV_LIL'],row['Chargingspeed_ICV_LIL'],row['Homedistnearestcharger_ICV_LIL'],row['Homechargeravailability_ICV_LIL']))
    v3 = (row['AV3'], V3(np.random.normal(),row['Purchaseprice_ICV_MEL'],row['Operationcost_ICV_MEL'],row['Yearlycost__ICV_MEL'],row['Range_ICV_MEL'],row['Acceleration_ICV_MEL'],row['Size_ICV_MEL'],row['CO2_ICV_MEL'],row['Charginginfrastructure_ICV_MEL'],row['Chargingspeed_ICV_MEL'],row['Homedistnearestcharger_ICV_MEL'],row['Homechargeravailability_ICV_MEL']))
    v4 = (row['AV4'], V4(np.random.normal(),row['Purchaseprice_ICV_STOR'],row['Operationcost_ICV_STOR'],row['Yearlycost__ICV_STOR'],row['Range_ICV_STOR'],row['Acceleration_ICV_STOR'],row['Size_ICV_STOR'],row['CO2_ICV_STOR'],row['Charginginfrastructure_ICV_STOR'],row['Chargingspeed_ICV_STOR'],row['Homedistnearestcharger_ICV_STOR'],row['Homechargeravailability_ICV_STOR']))
    v5 = (row['AV5'], V5(np.random.normal(),row['Purchaseprice_ICV_PREM'],row['Operationcost_ICV_PREM'],row['Yearlycost__ICV_PREM'],row['Range_ICV_PREM'],row['Acceleration_ICV_PREM'],row['Size_ICV_PREM'],row['CO2_ICV_PREM'],row['Charginginfrastructure_ICV_PREM'],row['Chargingspeed_ICV_PREM'],row['Homedistnearestcharger_ICV_PREM'],row['Homechargeravailability_ICV_PREM']))
    v6 = (row['AV6'], V6(np.random.normal(),row['Purchaseprice_ICV_LUK'],row['Operationcost_ICV_LUK'],row['Yearlycost__ICV_LUK'],row['Range_ICV_LUK'],row['Acceleration_ICV_LUK'],row['Size_ICV_LUK'],row['CO2_ICV_LUK'],row['Charginginfrastructure_ICV_LUK'],row['Chargingspeed_ICV_LUK'],row['Homedistnearestcharger_ICV_LUK'],row['Homechargeravailability_ICV_LUK']))

    v7 = (row['AV7'], V7(np.random.normal(),row['Purchaseprice_BEV_MINI'],row['Operationcost_BEV_MINI_NEW'],row['Yearlycost__BEV_MINI'],row['Range_BEV_MINI'],row['Acceleration_BEV_MINI'],row['Size_BEV_MINI'],row['CO2_BEV_MINI'],row['Charginginfrastructure_BEV_MINI'],row['Chargingspeed_BEV_MINI'],row['Homedistnearestcharger_BEV_MINI'],row['Homechargeravailability_BEV_MINI']))
    v8 = (row['AV8'], V8(np.random.normal(),row['Purchaseprice_BEV_LIL'],row['Operationcost_BEV_LIL_NEW'],row['Yearlycost__BEV_LIL'],row['Range_BEV_LIL'],row['Acceleration_BEV_LIL'],row['Size_BEV_LIL'],row['CO2_BEV_LIL'],row['Charginginfrastructure_BEV_LIL'],row['Chargingspeed_BEV_LIL'],row['Homedistnearestcharger_BEV_LIL'],row['Homechargeravailability_BEV_LIL']))
    v9 = (row['AV9'], V9(np.random.normal(),row['Purchaseprice_BEV_MEL'],row['Operationcost_BEV_MEL_NEW'],row['Yearlycost__BEV_MEL'],row['Range_BEV_MEL'],row['Acceleration_BEV_MEL'],row['Size_BEV_MEL'],row['CO2_BEV_MEL'],row['Charginginfrastructure_BEV_MEL'],row['Chargingspeed_BEV_MEL'],row['Homedistnearestcharger_BEV_MEL'],row['Homechargeravailability_BEV_MEL']))
    v10 = (row['AV10'], V10(np.random.normal(),row['Purchaseprice_BEV_STOR'],row['Operationcost_BEV_STOR_NEW'],row['Yearlycost__BEV_STOR'],row['Range_BEV_STOR'],row['Acceleration_BEV_STOR'],row['Size_BEV_STOR'],row['CO2_BEV_STOR'],row['Charginginfrastructure_BEV_STOR'],row['Chargingspeed_BEV_STOR'],row['Homedistnearestcharger_BEV_STOR'],row['Homechargeravailability_BEV_STOR']))
    v11 = (row['AV11'], V11(np.random.normal(),row['Purchaseprice_BEV_PREM'],row['Operationcost_BEV_PREM_NEW'],row['Yearlycost__BEV_PREM'],row['Range_BEV_PREM'],row['Acceleration_BEV_PREM'],row['Size_BEV_PREM'],row['CO2_BEV_PREM'],row['Charginginfrastructure_BEV_PREM'],row['Chargingspeed_BEV_PREM'],row['Homedistnearestcharger_BEV_PREM'],row['Homechargeravailability_BEV_PREM']))
    v12 = (row['AV12'], V12(np.random.normal(),row['Purchaseprice_BEV_LUK'],row['Operationcost_BEV_LUK_NEW'],row['Yearlycost__BEV_LUK'],row['Range_BEV_LUK'],row['Acceleration_BEV_LUK'],row['Size_BEV_LUK'],row['CO2_BEV_LUK'],row['Charginginfrastructure_BEV_LUK'],row['Chargingspeed_BEV_LUK'],row['Homedistnearestcharger_BEV_LUK'],row['Homechargeravailability_BEV_LUK']))

    v13 = (row['AV13'], V13(np.random.normal(),row['Purchaseprice_PHEV_MINI'],row['Operationcost_PHEV_MINI_NEW'],row['Yearlycost__PHEV_MINI'],row['Range_PHEV_MINI'],row['Acceleration_PHEV_MINI'],row['Size_PHEV_MINI'],row['CO2_PHEV_MINI'],row['Charginginfrastructure_PHEV_MINI'],row['Chargingspeed_PHEV_MINI'],row['Homedistnearestcharger_PHEV_MINI'],row['Homechargeravailability_PHEV_MIN']))
    v14 = (row['AV14'], V14(np.random.normal(),row['Purchaseprice_PHEV_LIL'],row['Operationcost_PHEV_LIL_NEW'],row['Yearlycost__PHEV_LIL'],row['Range_PHEV_LIL'],row['Acceleration_PHEV_LIL'],row['Size_PHEV_LIL'],row['CO2_PHEV_LIL'],row['Charginginfrastructure_PHEV_LIL'],row['Chargingspeed_PHEV_LIL'],row['Homedistnearestcharger_PHEV_LIL'],row['Homechargeravailability_PHEV_LIL']))
    v15 = (row['AV15'], V15(np.random.normal(),row['Purchaseprice_PHEV_MEL'],row['Operationcost_PHEV_MEL_NEW'],row['Yearlycost__PHEV_MEL'],row['Range_PHEV_MEL'],row['Acceleration_PHEV_MEL'],row['Size_PHEV_MEL'],row['CO2_PHEV_MEL'],row['Charginginfrastructure_PHEV_MEL'],row['Chargingspeed_PHEV_MEL'],row['Homedistnearestcharger_PHEV_MEL'],row['Homechargeravailability_PHEV_MEL']))
    v16 = (row['AV16'], V16(np.random.normal(),row['Purchaseprice_PHEV_STOR'],row['Operationcost_PHEV_STOR_NEW'],row['Yearlycost__PHEV_STOR'],row['Range_PHEV_STOR'],row['Acceleration_PHEV_STOR'],row['Size_PHEV_STOR'],row['CO2_PHEV_STOR'],row['Charginginfrastructure_PHEV_STOR'],row['Chargingspeed_PHEV_STOR'],row['Homedistnearestcharger_PHEV_STOR'],row['Homechargeravailability_PHEV_STO']))    
    v17 = (row['AV17'], V17(np.random.normal(),row['Purchaseprice_PHEV_PREM'],row['Operationcost_PHEV_PREM_NEW'],row['Yearlycost__PHEV_PREM'],row['Range_PHEV_PREM'],row['Acceleration_PHEV_PREM'],row['Size_PHEV_PREM'],row['CO2_PHEV_PREM'],row['Charginginfrastructure_PHEV_PREM'],row['Chargingspeed_PHEV_PREM'],row['Homedistnearestcharger_PHEV_PREM'],row['Homechargeravailability_PHEV_PRE']))
    v18 = (row['AV18'], V18(row['Purchaseprice_PHEV_LUK'],row['Operationcost_PHEV_LUK_NEW'],row['Yearlycost__PHEV_LUK'],row['Range_PHEV_LUK'],row['Acceleration_PHEV_LUK'],row['Size_PHEV_LUK'],row['CO2_PHEV_LUK'],row['Charginginfrastructure_PHEV_LUK'],row['Chargingspeed_PHEV_LUK'],row['Homedistnearestcharger_PHEV_LUK'],row['Homechargeravailability_PHEV_LUK']))    
    
    
    probVector_s3 = prob(mu, v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18)
    probMatrix_s3.append(probVector_s3)

Sample enumeration

In [116]:
tot = len(probMatrix_s3)
alt = len(mapping)
sampleEnum_s3 = {}
for i in range(len(mapping)):
    sampleEnum_s3[mapping[i]] = round((sum([x[i] for x in probMatrix_s3]) / tot) * 100,3)
df_res_s3 = pd.DataFrame([sampleEnum_s3])
df_res_s3.style.hide_index()

ICEV_1,ICEV_2,ICEV_3,ICEV_4,ICEV_5,ICEV_6,BEV_1,BEV_2,BEV_3,BEV_4,BEV_5,BEV_6,PHEV_1,PHEV_2,PHEV_3,PHEV_4,PHEV_5,PHEV_6
6.906000,10.102000,16.064000,7.420000,4.088000,0.823000,5.460000,5.801000,15.100000,9.300000,3.438000,0.660000,1.981000,2.456000,6.755000,2.690000,0.747000,0.208000


#### Scenario 4
Higher oil prices (EU’s climate policy, etc.) 

Calculating probabilities

In [117]:
mu = 1

probMatrix_s4 = []
for index, row in df_s3.iterrows():
    v1 = (row['AV1'], V1(np.random.normal(),row['Purchaseprice_ICV_MINI'],row['Operationcost_ICV_MINI_NEW'],row['Yearlycost__ICV_MINI'],row['Range_ICV_MINI'],row['Acceleration_ICV_MINI'],row['Size_ICV1'],row['CO2_ICV_MINI'],row['Charginginfrastructure_ICV_MINI'],row['Chargingspeed_ICV_MINI'],row['Homedistnearestcharger_ICV_MINI'],row['Homechargeravailability_ICV_MINI']))
    v2 = (row['AV2'], V2(np.random.normal(),row['Purchaseprice_ICV_LIL'],row['Operationcost_ICV_LIL_NEW'],row['Yearlycost__ICV_LIL'],row['Range_ICV_LIL'],row['Acceleration_ICV_LIL'],row['Size_ICV_LIL'],row['CO2_ICV_LIL'],row['Charginginfrastructure_ICV_LIL'],row['Chargingspeed_ICV_LIL'],row['Homedistnearestcharger_ICV_LIL'],row['Homechargeravailability_ICV_LIL']))
    v3 = (row['AV3'], V3(np.random.normal(),row['Purchaseprice_ICV_MEL'],row['Operationcost_ICV_MEL_NEW'],row['Yearlycost__ICV_MEL'],row['Range_ICV_MEL'],row['Acceleration_ICV_MEL'],row['Size_ICV_MEL'],row['CO2_ICV_MEL'],row['Charginginfrastructure_ICV_MEL'],row['Chargingspeed_ICV_MEL'],row['Homedistnearestcharger_ICV_MEL'],row['Homechargeravailability_ICV_MEL']))
    v4 = (row['AV4'], V4(np.random.normal(),row['Purchaseprice_ICV_STOR'],row['Operationcost_ICV_STOR_NEW'],row['Yearlycost__ICV_STOR'],row['Range_ICV_STOR'],row['Acceleration_ICV_STOR'],row['Size_ICV_STOR'],row['CO2_ICV_STOR'],row['Charginginfrastructure_ICV_STOR'],row['Chargingspeed_ICV_STOR'],row['Homedistnearestcharger_ICV_STOR'],row['Homechargeravailability_ICV_STOR']))
    v5 = (row['AV5'], V5(np.random.normal(),row['Purchaseprice_ICV_PREM'],row['Operationcost_ICV_PREM_NEW'],row['Yearlycost__ICV_PREM'],row['Range_ICV_PREM'],row['Acceleration_ICV_PREM'],row['Size_ICV_PREM'],row['CO2_ICV_PREM'],row['Charginginfrastructure_ICV_PREM'],row['Chargingspeed_ICV_PREM'],row['Homedistnearestcharger_ICV_PREM'],row['Homechargeravailability_ICV_PREM']))
    v6 = (row['AV6'], V6(np.random.normal(),row['Purchaseprice_ICV_LUK'],row['Operationcost_ICV_LUK_NEW'],row['Yearlycost__ICV_LUK'],row['Range_ICV_LUK'],row['Acceleration_ICV_LUK'],row['Size_ICV_LUK'],row['CO2_ICV_LUK'],row['Charginginfrastructure_ICV_LUK'],row['Chargingspeed_ICV_LUK'],row['Homedistnearestcharger_ICV_LUK'],row['Homechargeravailability_ICV_LUK']))

    v7 = (row['AV7'], V7(np.random.normal(),row['Purchaseprice_BEV_MINI'],row['Operationcost_BEV_MINI'],row['Yearlycost__BEV_MINI'],row['Range_BEV_MINI'],row['Acceleration_BEV_MINI'],row['Size_BEV_MINI'],row['CO2_BEV_MINI'],row['Charginginfrastructure_BEV_MINI'],row['Chargingspeed_BEV_MINI'],row['Homedistnearestcharger_BEV_MINI'],row['Homechargeravailability_BEV_MINI']))
    v8 = (row['AV8'], V8(np.random.normal(),row['Purchaseprice_BEV_LIL'],row['Operationcost_BEV_LIL'],row['Yearlycost__BEV_LIL'],row['Range_BEV_LIL'],row['Acceleration_BEV_LIL'],row['Size_BEV_LIL'],row['CO2_BEV_LIL'],row['Charginginfrastructure_BEV_LIL'],row['Chargingspeed_BEV_LIL'],row['Homedistnearestcharger_BEV_LIL'],row['Homechargeravailability_BEV_LIL']))
    v9 = (row['AV9'], V9(np.random.normal(),row['Purchaseprice_BEV_MEL'],row['Operationcost_BEV_MEL'],row['Yearlycost__BEV_MEL'],row['Range_BEV_MEL'],row['Acceleration_BEV_MEL'],row['Size_BEV_MEL'],row['CO2_BEV_MEL'],row['Charginginfrastructure_BEV_MEL'],row['Chargingspeed_BEV_MEL'],row['Homedistnearestcharger_BEV_MEL'],row['Homechargeravailability_BEV_MEL']))
    v10 = (row['AV10'], V10(np.random.normal(),row['Purchaseprice_BEV_STOR'],row['Operationcost_BEV_STOR'],row['Yearlycost__BEV_STOR'],row['Range_BEV_STOR'],row['Acceleration_BEV_STOR'],row['Size_BEV_STOR'],row['CO2_BEV_STOR'],row['Charginginfrastructure_BEV_STOR'],row['Chargingspeed_BEV_STOR'],row['Homedistnearestcharger_BEV_STOR'],row['Homechargeravailability_BEV_STOR']))
    v11 = (row['AV11'], V11(np.random.normal(),row['Purchaseprice_BEV_PREM'],row['Operationcost_BEV_PREM'],row['Yearlycost__BEV_PREM'],row['Range_BEV_PREM'],row['Acceleration_BEV_PREM'],row['Size_BEV_PREM'],row['CO2_BEV_PREM'],row['Charginginfrastructure_BEV_PREM'],row['Chargingspeed_BEV_PREM'],row['Homedistnearestcharger_BEV_PREM'],row['Homechargeravailability_BEV_PREM']))
    v12 = (row['AV12'], V12(np.random.normal(),row['Purchaseprice_BEV_LUK'],row['Operationcost_BEV_LUK'],row['Yearlycost__BEV_LUK'],row['Range_BEV_LUK'],row['Acceleration_BEV_LUK'],row['Size_BEV_LUK'],row['CO2_BEV_LUK'],row['Charginginfrastructure_BEV_LUK'],row['Chargingspeed_BEV_LUK'],row['Homedistnearestcharger_BEV_LUK'],row['Homechargeravailability_BEV_LUK']))

    v13 = (row['AV13'], V13(np.random.normal(),row['Purchaseprice_PHEV_MINI'],row['Operationcost_PHEV_MINI'],row['Yearlycost__PHEV_MINI'],row['Range_PHEV_MINI'],row['Acceleration_PHEV_MINI'],row['Size_PHEV_MINI'],row['CO2_PHEV_MINI'],row['Charginginfrastructure_PHEV_MINI'],row['Chargingspeed_PHEV_MINI'],row['Homedistnearestcharger_PHEV_MINI'],row['Homechargeravailability_PHEV_MIN']))
    v14 = (row['AV14'], V14(np.random.normal(),row['Purchaseprice_PHEV_LIL'],row['Operationcost_PHEV_LIL'],row['Yearlycost__PHEV_LIL'],row['Range_PHEV_LIL'],row['Acceleration_PHEV_LIL'],row['Size_PHEV_LIL'],row['CO2_PHEV_LIL'],row['Charginginfrastructure_PHEV_LIL'],row['Chargingspeed_PHEV_LIL'],row['Homedistnearestcharger_PHEV_LIL'],row['Homechargeravailability_PHEV_LIL']))
    v15 = (row['AV15'], V15(np.random.normal(),row['Purchaseprice_PHEV_MEL'],row['Operationcost_PHEV_MEL'],row['Yearlycost__PHEV_MEL'],row['Range_PHEV_MEL'],row['Acceleration_PHEV_MEL'],row['Size_PHEV_MEL'],row['CO2_PHEV_MEL'],row['Charginginfrastructure_PHEV_MEL'],row['Chargingspeed_PHEV_MEL'],row['Homedistnearestcharger_PHEV_MEL'],row['Homechargeravailability_PHEV_MEL']))
    v16 = (row['AV16'], V16(np.random.normal(),row['Purchaseprice_PHEV_STOR'],row['Operationcost_PHEV_STOR'],row['Yearlycost__PHEV_STOR'],row['Range_PHEV_STOR'],row['Acceleration_PHEV_STOR'],row['Size_PHEV_STOR'],row['CO2_PHEV_STOR'],row['Charginginfrastructure_PHEV_STOR'],row['Chargingspeed_PHEV_STOR'],row['Homedistnearestcharger_PHEV_STOR'],row['Homechargeravailability_PHEV_STO']))    
    v17 = (row['AV17'], V17(np.random.normal(),row['Purchaseprice_PHEV_PREM'],row['Operationcost_PHEV_PREM'],row['Yearlycost__PHEV_PREM'],row['Range_PHEV_PREM'],row['Acceleration_PHEV_PREM'],row['Size_PHEV_PREM'],row['CO2_PHEV_PREM'],row['Charginginfrastructure_PHEV_PREM'],row['Chargingspeed_PHEV_PREM'],row['Homedistnearestcharger_PHEV_PREM'],row['Homechargeravailability_PHEV_PRE']))
    v18 = (row['AV18'], V18(row['Purchaseprice_PHEV_LUK'],row['Operationcost_PHEV_LUK'],row['Yearlycost__PHEV_LUK'],row['Range_PHEV_LUK'],row['Acceleration_PHEV_LUK'],row['Size_PHEV_LUK'],row['CO2_PHEV_LUK'],row['Charginginfrastructure_PHEV_LUK'],row['Chargingspeed_PHEV_LUK'],row['Homedistnearestcharger_PHEV_LUK'],row['Homechargeravailability_PHEV_LUK']))     
    
    
    probVector_s4 = prob(mu, v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18)
    probMatrix_s4.append(probVector_s4)

Sample enumeration

In [118]:
tot = len(probMatrix_s4)
alt = len(mapping)
sampleEnum_s4 = {}
for i in range(len(mapping)):
    sampleEnum_s4[mapping[i]] = round((sum([x[i] for x in probMatrix_s4]) / tot) * 100,3)
df_res_s4 = pd.DataFrame([sampleEnum_s4])
df_res_s4.style.hide_index()

ICEV_1,ICEV_2,ICEV_3,ICEV_4,ICEV_5,ICEV_6,BEV_1,BEV_2,BEV_3,BEV_4,BEV_5,BEV_6,PHEV_1,PHEV_2,PHEV_3,PHEV_4,PHEV_5,PHEV_6
5.550000,8.003000,13.055000,5.526000,3.085000,0.660000,6.359000,6.901000,17.887000,10.596000,4.037000,0.717000,2.350000,2.894000,7.815000,3.279000,0.962000,0.325000


#### Consolidation of results

In [119]:
rn_cols = {}
for i in range(18):
    rn_cols.update({i:mapping[i]})

#rn_cols

In [120]:
df_test = df_test.rename(columns=rn_cols)
df_test

,ICEV_1,ICEV_2,ICEV_3,ICEV_4,ICEV_5,ICEV_6,BEV_1,BEV_2,BEV_3,BEV_4,BEV_5,BEV_6,PHEV_1,PHEV_2,PHEV_3,PHEV_4,PHEV_5,PHEV_6
CHOICE18,1.000000,2.00000,3.000000,4.000000,5.000000,6.000000,7.00000,8.00000,9.00000,10.000000,11.000000,12.000000,13.000000,14.000000,15.000000,16.000000,17.000000,18.000000
share,6.496579,9.22531,15.223452,6.648644,3.649573,0.789896,5.29695,6.68666,17.39461,9.195742,2.787869,0.578694,2.293655,2.830109,7.235786,2.830109,0.633606,0.202754


In [121]:
df_test.drop(labels='CHOICE18', axis=0,inplace=True)
df_test

,ICEV_1,ICEV_2,ICEV_3,ICEV_4,ICEV_5,ICEV_6,BEV_1,BEV_2,BEV_3,BEV_4,BEV_5,BEV_6,PHEV_1,PHEV_2,PHEV_3,PHEV_4,PHEV_5,PHEV_6
share,6.496579,9.22531,15.223452,6.648644,3.649573,0.789896,5.29695,6.68666,17.39461,9.195742,2.787869,0.578694,2.293655,2.830109,7.235786,2.830109,0.633606,0.202754


In [122]:
df_test['Label'] = 'Observed choices'
df_res['Label'] = 'Survey data'
df_res_s1['Label'] = 'Scenario 1'
df_res_s2['Label'] = 'Scenario 2'
df_res_s3['Label'] = 'Scenario 3'
df_res_s4['Label'] = 'Scenario 4'

In [123]:
df_result = pd.concat([df_test,df_res,df_res_s1,df_res_s2,df_res_s3,df_res_s4]).reset_index()

In [125]:
df_result[['Label','ICEV_1','ICEV_2','ICEV_3','ICEV_4','ICEV_5','ICEV_6','BEV_1','BEV_2','BEV_3','BEV_4','BEV_5','BEV_6','PHEV_1','PHEV_2','PHEV_3','PHEV_4','PHEV_5','PHEV_6']].transpose()

,0,1,2,3,4,5
Label,Observed choices,Survey data,Scenario 1,Scenario 2,Scenario 3,Scenario 4
ICEV_1,6.49658,6.223,5.632,6.344,6.906,5.55
ICEV_2,9.22531,9.271,7.479,9.306,10.102,8.003
ICEV_3,15.2235,15.207,11.265,15.288,16.064,13.055
ICEV_4,6.64864,6.768,4.393,6.725,7.42,5.526
ICEV_5,3.64957,3.549,2.142,3.661,4.088,3.085
ICEV_6,0.789896,0.779,0.5,0.899,0.823,0.66
BEV_1,5.29695,5.748,5.8,5.661,5.46,6.359
BEV_2,6.68666,6.285,7.406,6.134,5.801,6.901
BEV_3,17.3946,16.088,19.543,15.539,15.1,17.887
